# Satellite Acquisition 
## (via the parallel frequency space search algorithm)
### (It still doesn't work)

**BEFORE RUNNING THIS CODE: **Download the launch 12 GPS data [here](https://github.com/psas/Launch-12/blob/gh-pages/data/GPS/JGPS%40-32.041913222), and place it in the */resources* folder.

This notebook demonstrates the satellite acquisition proccess, in which we determine which satellites are overhead. This code implements the parallel code phase search algorithm, which is the most complex of those available, but is also the fastest since it returns a result in just one iteration. It is possible that we will need to use a different algorithm when the code is running on an embedded platform.

As noted in the L-12 GPS data README, the sampling frequency is 4.092 MHz. and the IF is 0. Also note that the L1 signal hits the antenna at the 1.57542 GHz frequency (10.23 MHz x 154). This is dealt with by the hardware frontend, so we only have to concern ourselves with the 1.023 MHz BPSK modulated C/A code.

**Note: we should only perform this acquisition method on ~20ms sections of data, see book for details**

## Preamble

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#import peakutils
import struct
import math
import scipy.stats as sp

##Grabs the GoldCode module from the root of the repo
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from GoldCode import GoldCode
from GPSData import IQData


## Import data File

- Information on file "JGPS@-32.041913222" can be found at: https://github.com/psas/Launch-12/tree/gh-pages/data/GPS

In [ ]:
# Need these to pass to importFile module
fs = 4.092*10**6
SampleLength = 0.001
BytesToSkip = 2046

data = IQData()
data.importFile('../resources/JGPS@-32.041913222', fs, SampleLength, BytesToSkip)

## Check out the imported data

We will plot the first 100 samples to see what the signal looks like.

In [ ]:
plt.plot(data.t, np.real(data.CData))
plt.plot(data.t, np.imag(data.CData))

# Set xaxis so first 100 samples are shown
xmin,xmax = plt.xlim()
plt.xlim(xmin,xmin + 100/fs)

plt.title("Plot of CData (Real and Imaginary parts)")
plt.xlabel("t (microseconds)")
plt.show()

# Get a Gold Code


In [ ]:

def getSat(select):
    # Create list of C/A code Taps, for simpler sat selection",
    #sat = [(2,6),(3,7),(4,8),(5,9),(1,9),(2,6),(1,8),(2,9),(3,10),(2,3),(3,4),(5,6),(6,7),(7,8),(8,9),(9,10),(1,4),(2,5),(3,6),(4,7),(5,8),(6,9),(1,3),(4,6),(5,7),(6,8),(7,9),(8,10),(1,6),(2,7),(3,8),(4,9),(5,10),(4,10),(1,7),(2,8),(4,10)]
    sat = [(1,5),(2,6),(3,7),(4,8),(0,8),(1,5),(0,7),(1,8),(2,9),(1,2),(2,3),(4,5),(5,6),(6,7),(7,8),(8,9),(0,3),(1,4),(2,5),(3,6),(4,7),(5,8),(0,2),(3,5),(4,6),(5,7),(6,8),(7,9),(0,5),(1,6),(2,7),(3,8),(4,9),(3,9),(0,6),(1,7),(3,9)]

    # Create Code Generator object for chosen Satellite
    CodeGen = GoldCode(sat[select - 1]) # Index starts at zero

    # Generate CA Code
    CACode = CodeGen.getCode(1023, samplesPerChip = 4)

    print("Satellite chosen: %d, with tap: %s" %(select,str(sat[select - 1])))

    # Repeat entire array for each ms of data sampled
    CACodeSampled = np.tile(CACode, data.sampleTime / 1e-3)
    print(len(CACodeSampled))
    return CACodeSampled

# Multiply Signal and Code

In [ ]:
nfft = data.Nsamples

# Generate frequency lists for plotting fft
# Can use f for regular fft and fshift for shifted version
fs_kHz = data.sampleFreq#/1000 #Make easier to read
f = np.linspace (0,fs_kHz,nfft, endpoint=False)
fshift = np.linspace(-fs_kHz/2,fs_kHz/2,nfft,endpoint=False)

#fstep = fs/N
fax = np.linspace(0, data.sampleFreq * 2, nfft)

from collections import deque
cadeque = deque()

#for i in CACodeSampled:
#    cadeque.append(i)
    
def mixnplot(CACodeSampled, compData, nfft, f, lab, plots = False):
    
    mixedData = compData * CACodeSampled
    fftData = np.fft.fft(mixedData, nfft)

    psd = np.abs(fftData) ** 2

    psddb = (10 * np.log10(psd))
    
    #Set plots to True if you want to look over 1023 plots manually
    if plots:
        psddb_norm = psddb - np.max(psddb)
        
        plt.title("Iter: " + str(lab))
        plt.plot(fax/1000, psddb_norm)
        savedir = './plot2/' + str(lab)
        plt.savefig(savedir)
        plt.clf()
    
    #Return the ratio of the highest peak
    return psddb.max()/psddb.mean()

# Get 32 CA codes, mix them, and plot the peak ratios

In [ ]:
for s in range(32):
    code = getSat(s)
    
    codeq = deque()

    for i in code:
        codeq.append(i)
    
    ratios = []
    for i in range(1023):
        ratio = mixnplot(codeq, data.CData, nfft, f, str(i))
        ratios.append(ratio)
        codeq.rotate(-4)
        #i += 1
        
        #print(ratio)
        #if i % 10 == 0:
        #   print(str(i*100/1023) + '% complete')
    
    plt.title("SV " + str(s + 1))
    plt.plot(ratios)
    plt.show()
        
    

In [ ]:
plt.plot(ratios)
plt.show()